In [2]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv("../data/A_to_Z_Flowers_indicateurs.csv")

# Afficher les noms des colonnes
print(df.columns)


Index(['Name', 'Desc', 'SunNeeds', 'WaterNeeds', 'Maintenance', 'Type de Sol',
       'SunNeeds_Score', 'WaterNeeds_Score', 'Maintenance_score', 'saison',
       'plant_categories', 'min_height_cm', 'max_height_cm', 'min_ph',
       'max_ph'],
      dtype='object')


In [ ]:
import openai

openai_api_key = ""

def chatgpt_recommendation(prompt):
    """
    Envoie le prompt à ChatGPT pour obtenir une description de la plante idéale.
    """
    client = openai.OpenAI(api_key=openai_api_key)  # Définition explicite de la clé API
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "Tu es un expert en botanique et tu aides à recommander des plantes."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Test de la fonction
prompt_utilisateur = "Je veux une plante d’intérieur qui a besoin de peu de lumière et d’entretien."
reponse_ai = chatgpt_recommendation(prompt_utilisateur)

print("🔹 Plante suggérée par ChatGPT :", reponse_ai)


🔹 Plante suggérée par ChatGPT : Une plante parfaite pour vous serait le zz plant (Zamioculcas zamiifolia). Cette plante d'intérieur est connue pour sa tolérance à des conditions de faible luminosité et son besoin minimal en arrosage. Le zz plant est également peu exigeant en termes de soins, ce qui en fait un choix idéal pour les personnes qui n'ont pas beaucoup de temps à consacrer à l'entretien de leurs plantes. Ses feuilles lisses et brillantes apporteront une touche de verdure à votre intérieur tout en étant facile à entretenir.


In [21]:
def filtrer_plante(reponse_ai, df):
    """
    Utilise la description générée par ChatGPT pour filtrer les plantes dans notre dataset avec un système de score.
    """
    filtered_df = df.copy()
    
    # Définition des mots-clés pour les critères
    criteres = {
        "lumière": {
            "ombre": ["Ombre"],
            "mi-ombre": ["Mi-ombre"],
            "soleil": ["Plein soleil"]
        },
        "eau": {
            "peu d'eau": ["Faible"],
            "moyenne": ["Moyenne"],
            "beaucoup d'eau": ["Elevée"]
        },
        "entretien": {
            "peu d’entretien": ["Faible"],
            "entretien moyen": ["Moyen"],
            "entretien élevé": ["Elevé"]
        }
    }
    
    # Création d'un score pour classer les résultats
    filtered_df["score"] = 0
    
    # 🔍 Analyse des critères mentionnés dans la réponse de ChatGPT
    for critere, valeurs in criteres.items():
        for mot_cle, valeurs_associees in valeurs.items():
            if mot_cle in reponse_ai.lower():
                for valeur in valeurs_associees:
                    if critere == "lumière":
                        filtered_df.loc[filtered_df["SunNeeds"].str.contains(valeur, na=False), "score"] += 1
                    elif critere == "eau":
                        filtered_df.loc[filtered_df["WaterNeeds"].str.contains(valeur, na=False), "score"] += 1
                    elif critere == "entretien":
                        filtered_df.loc[filtered_df["Maintenance"].str.contains(valeur, na=False), "score"] += 1
    
    # 🌷 Ajout de la saison si mentionnée dans la réponse
    for saison in ["Printemps", "Été", "Automne", "Hiver"]:
        if saison.lower() in reponse_ai.lower():
            filtered_df.loc[filtered_df["saison"].str.contains(saison, na=False), "score"] += 1

    # 🌱 Ajout du type de sol si mentionné
    for sol in ["Argile", "Limon", "Sable"]:
        if sol.lower() in reponse_ai.lower():
            filtered_df.loc[filtered_df["Type de Sol"].str.contains(sol, na=False), "score"] += 1

    # 🎯 Trier les plantes par score décroissant et retourner les meilleures suggestions
    filtered_df = filtered_df.sort_values(by="score", ascending=False)
    result = filtered_df[["Name", "Desc", "SunNeeds", "WaterNeeds", "Maintenance", "saison", "Type de Sol", "score"]].head(3)
    
    return result if not result.empty else "❌ Aucune plante ne correspond aux critères."


# Tester la fonction avec la réponse de ChatGPT
plantes_recommandees = filtrer_plante(reponse_ai, df)
print(plantes_recommandees)


                     Name                                               Desc  \
0         Nelumbo (Lotus)  Nelumbo est un genre de plantes aquatiques ne ...   
188     Weigela (Weigela)  Weigela est un genre d’environ 10 espèces d’ar...   
194  Calla (Zantedeschia)  Le Calla, aussi appelé Arum, est une plante él...   

                   SunNeeds WaterNeeds Maintenance                   saison  \
0              Plein soleil    Moyenne      Faible           Printemps, Été   
188  Plein soleil, Mi-ombre    Moyenne      Faible  Printemps, Automne, Été   
194  Plein soleil, Mi-ombre    Moyenne      Faible           Printemps, Été   

    Type de Sol  score  
0        Argile      0  
188       Limon      0  
194       Limon      0  


In [ ]:
def recommander_plantes(prompt, df):
    """
    1. Envoie le prompt utilisateur à ChatGPT pour définir le type de plante idéale.
    2. Filtre les données pour trouver la meilleure correspondance.
    """
    print("🔄 Analyse de votre demande...")
    
    # Étape 1 : Demander une description à ChatGPT
    reponse_ai = chatgpt_recommendation(prompt)
    print(f"🔹 Plante idéale selon ChatGPT : {reponse_ai}")
    
    # Étape 2 : Trouver la meilleure correspondance dans les données
    plantes_recommandees = filtrer_plante(reponse_ai, df)
    
    # Étape 3 : Afficher les résultats
    if not plantes_recommandees.empty:
        print("\n🌱 Plantes recommandées :")
        print(plantes_recommandees)
    else:
        print("❌ Aucune plante ne correspond aux critères.")
    
    return plantes_recommandees

# Tester avec un prompt
prompt_utilisateur = "Je cherche une plante qui demande peu d’entretien et pousse à l’ombre."
recommander_plantes(prompt_utilisateur, df) 

🔄 Analyse de votre demande...
🔹 Plante idéale selon ChatGPT : Je vous recommande d'opter pour une plante d'intérieur telle que le spathiphyllum (aussi appelé "fleur de lune") ou le calathea. Ces plantes sont adaptées à une faible luminosité et nécessitent peu d'entretien. Assurez-vous de les arroser régulièrement et de les garder à l'écart des courants d'air pour qu'elles s'épanouissent dans votre espace ombragé.

🌱 Plantes recommandées :
                             Name  \
0                 Nelumbo (Lotus)   
1               Kariota (Caryota)   
2  Lierre grimpant (Hedera helix)   

                                                Desc         SunNeeds  \
0  Nelumbo est un genre de plantes aquatiques ne ...     Plein soleil   
1  Le Kariota, aussi appelé Palmier queue de pois...  Mi-ombre, Ombre   
2  Le Lierre grimpant est une plante vivace résis...  Mi-ombre, Ombre   

  Maintenance  
0      Faible  
1       Moyen  
2      Faible  


,Name,Desc,SunNeeds,Maintenance
0,Nelumbo (Lotus),Nelumbo est un genre de plantes aquatiques ne ...,Plein soleil,Faible
1,Kariota (Caryota),"Le Kariota, aussi appelé Palmier queue de pois...","Mi-ombre, Ombre",Moyen
2,Lierre grimpant (Hedera helix),Le Lierre grimpant est une plante vivace résis...,"Mi-ombre, Ombre",Faible
